# 部署 Single Shot Multibox Detector(SSD) 模型

参考：[deploy_ssd_gluoncv](https://tvm.apache.org/docs/how_to/deploy_models/deploy_ssd_gluoncv.html)

使用 GluonCV 预训练 SSD 模型，并将其转换为 Relay IR。

In [1]:
import tvm
from matplotlib import pyplot as plt
from tvm import relay
from tvm.contrib import graph_executor
from tvm.contrib.download import download_testdata
from gluoncv import model_zoo, data, utils

## 初始和设置参数

```{note}
:class: alert alert-info

现在支持在 CPU 和 GPU 上编译 SSD。

为了在 CPU 上获得最佳的推理性能，根据你的设备改变目标参数，按照 {ref}`tune_relay_x86` 来调优 x86 CPU，按照 {ref}`tune_relay_arm` 来调优 ARM CPU。

为了在 Intel graphics 上获得最佳的推理性能，将目标参数改为 `opencl -device=intel_graphics`。但是当在 Mac 上使用 Intel graphics 时，target 需要设置为 `opencl`，只是因为 Mac 上不支持 Intel subgroup 扩展。

为了在基于 ``cuda`` 的 GPU 上获得最佳的推理性能，将目标参数改为 ``cuda``；对于基于 ``opencl`` 的 GPU，根据你的设备更改目标参数为 ``opencl``。
```

In [2]:
supported_model = [
    "ssd_512_resnet50_v1_voc",
    "ssd_512_resnet50_v1_coco",
    "ssd_512_resnet101_v2_voc",
    "ssd_512_mobilenet1.0_voc",
    "ssd_512_mobilenet1.0_coco",
    "ssd_300_vgg16_atrous_voc",
    "ssd_512_vgg16_atrous_coco",
]

model_name = supported_model[0]
dshape = (1, 3, 512, 512)

下载并预处理演示图像。

In [3]:
im_fname = download_testdata(
    "https://github.com/dmlc/web-data/blob/main/" + "gluoncv/detection/street_small.jpg?raw=true",
    "street_small.jpg",
    module="data",
)
x, img = data.transforms.presets.ssd.load_test(im_fname, short=512)

为 CPU 转换和编译模型。

In [4]:
block = model_zoo.get_model(model_name, pretrained=True)


def build(target):
    mod, params = relay.frontend.from_mxnet(block, {"data": dshape})
    with tvm.transform.PassContext(opt_level=3):
        lib = relay.build(mod, target, params=params)
    return lib

/media/workspace/anaconda3/envs/mx/lib/python3.10/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]




```{admonition} 创建 TVM 运行时并进行推理
:class: alert alert-info

如果你在 cmake 期间设定 `-DUSE_THRUST=ON` 启用了 thrust，则使用 ``target = "cuda -libs"`` 来启用基于 thrust 的排序。
```

In [ ]:
def run(lib, dev):
    # Build TVM runtime
    m = graph_executor.GraphModule(lib["default"](dev))
    tvm_input = tvm.nd.array(x.asnumpy(), device=dev)
    m.set_input("data", tvm_input)
    # execute
    m.run()
    # get outputs
    class_IDs, scores, bounding_boxs = m.get_output(0), m.get_output(1), m.get_output(2)
    return class_IDs, scores, bounding_boxs


for target in ["llvm", "cuda"]:
    dev = tvm.device(target, 0)
    if dev.exist:
        lib = build(target)
        class_IDs, scores, bounding_boxs = run(lib, dev)

显示结果：

In [ ]:
ax = utils.viz.plot_bbox(
    img,
    bounding_boxs.numpy()[0],
    scores.numpy()[0],
    class_IDs.numpy()[0],
    class_names=block.classes,
)
plt.show()

In [ ]:
import mxnet as mx

inp = mx.nd.array(x.asnumpy())
block.summary(inp)

In [ ]:
36927598/(2**20)